## Introduction to Iceberg Architecture

In [2]:
!pip3 install -r requirements.txt

  Cloning https://github.com/fastforwardlabs/cmlbootstrap to /tmp/pip-install-bjic7697/cmlbootstrap_cacac7a43b3246cab40e19e8b837b239
  Running command git clone -q https://github.com/fastforwardlabs/cmlbootstrap /tmp/pip-install-bjic7697/cmlbootstrap_cacac7a43b3246cab40e19e8b837b239


#### Launching a Spark Session with Iceberg

In [3]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/09/04 02:25:52 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 02:25:52 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/04 02:25:53 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|        cde_workshop|
|             cdedemo|
|        cdp_overview|
|      ceht_open_data|
|        ceht_scratch|
| ceht_transportation|
|        cgsifacebook|
+--------------------+
only showing top 20 rows



In [4]:
spark.sparkContext.getConf().getAll()

23/09/04 02:26:07 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/04 02:26:07 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


[('spark.eventLog.enabled', 'true'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.kubernetes.driver.pod.name', 't0u1wnfhww2zkauf'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.driver.bindAddress', '100.100.216.41'),
 ('spark.jars',
  '/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar,/opt/spark/optional-lib/iceberg-hive-runtime.jar,/opt/spark/optional-lib/iceberg-spark-runtime.jar'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.driver.port', '33261'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.kubernetes.executor.podNamePrefix', 'cdsw-t0u1wnfhww2zkauf'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.ui.port', '20049'),
 ('spark.yarn.access.hadoopFileSystems',
  '

### Iceberg Architecture

![alt text](../img/iceberg-metadata.png)

#### Iceberg Catalog

Iceberg comes with catalogs that enable SQL commands to manage tables and load them by name. Catalogs are configured using properties under spark.sql.catalog.(catalog_name).

In [5]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [6]:
# Create a new database
#spark.sql("DROP DATABASE IF EXISTS spark_catalog.lakehouse")
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse")
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [7]:
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [8]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|lakehouse|
+-------------+---------+



#### Create an Iceberg Table with Spark SQL

In [9]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffees_table PURGE")

23/09/04 02:26:08 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...
23/09/04 02:26:11 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-16/pods. Message: pods "cdsw-t0u1wnfhww2zkauf-exec-1" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-t0u1wnfhww2zkauf-exec-1, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-t0u1wnfhww2zkauf-exec-1" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.Operatio

DataFrame[]

# TEST 1 with COW

In [10]:
spark.sql("CREATE TABLE IF NOT EXISTS coffees_table (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
          USING ICEBERG\
          PARTITIONED BY (months(coffee_sale_ts))\
          TBLPROPERTIES ('write.delete.mode'='copy-on-write',\
                          'write.update.mode'='copy-on-write',\
                          'write.merge.mode'='copy-on-write',\
                          'format-version' = '2')")

DataFrame[]

#### Verify that a Metadata JSON file has been created under the Metadata directory

In [11]:
metadata_path = "warehouse/tablespace/external/hive/lakehouse.db/coffees_table"

In [12]:
import boto3

s3 = boto3.resource('s3')

my_bucket = s3.Bucket("go01-demo")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path):
    #print(object_summary.key)
    metadata_file = object_summary.key
    
print("Metadata File Path: {}".format(metadata_file))

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table_3/metadata/snap-8731908966971168016-1-6f505010-2ec8-4a68-a802-4854d523bd19.avro


In [13]:
import pandas as pd
spark.read.option("multiline","true").json("s3a://go01-demo/" + "warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata").toPandas()

,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693794404641,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, copy-on-write, copy-on-write, co...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],63bffafe-89b6-4dc7-91fa-e0698c4347eb


![alt text](../img/s3_metadata.png)

#### Notice that no snapshots or other files have been created as data has not yet been inserted.

In [14]:
spark.sql("SELECT * FROM lakehouse.coffees_table.history").show()

+---------------+-----------+---------+-------------------+
|made_current_at|snapshot_id|parent_id|is_current_ancestor|
+---------------+-----------+---------+-------------------+
+---------------+-----------+---------+-------------------+



In [15]:
spark.sql("SELECT * FROM lakehouse.coffees_table.snapshots;").show()

+------------+-----------+---------+---------+-------------+-------+
|committed_at|snapshot_id|parent_id|operation|manifest_list|summary|
+------------+-----------+---------+---------+-------------+-------+
+------------+-----------+---------+---------+-------------+-------+



In [16]:
spark.sql("SELECT * FROM lakehouse.coffees_table.files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [17]:
spark.sql("SELECT * FROM lakehouse.coffees_table.manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+



In [18]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [19]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|reference_snapshot_id|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+-------

### Table Insert

In [20]:
from pyspark.sql.functions import date_format

In [21]:
#Coffee_id = 1, Coffee_size = venti, coffee_sale_ts = 2023-09-01

spark.sql("INSERT INTO lakehouse.coffees_table VALUES (1, 'venti', cast(date_format('2023-09-01 10:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

#### Data has been added to the data folder

In [22]:
QUERY = "select h.made_current_at,\
            s.operation,\
            h.snapshot_id,\
            h.is_current_ancestor,\
            s.summary['spark.app.id']\
        from lakehouse.coffees_table.history h\
        join lakehouse.coffees_table.snapshots s\
            on h.snapshot_id = s.snapshot_id\
            order by made_current_at;"

In [23]:
spark.sql(QUERY).toPandas()

23/09/04 02:26:55 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


,made_current_at,operation,snapshot_id,is_current_ancestor,summary[spark.app.id]
0,2023-09-04 02:26:53.905,append,7240979918569329194,True,spark-application-1693794355236


#### Notice there are now two json files and two avro files. 

The first json file is the metadata file created when the table was created. This is the metata file prefixed by 00000. The second json file is the new metadata file reflecting the insert of one row. This is the metadata file prefixed by 00001.

The avro file with the "snap" prefix is the manifest list. The other avro file created is the corresponding manifest file.

In [24]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")
for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

Current Metadata Files: 

warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-bebe7ef5-5289-4cad-981d-6bb461634c57.metadata.json

warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-1ba964c5-4142-4d48-a738-fb31d408c861.metadata.json

warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/ec4cbc6d-78e5-47ca-b0cd-480abd39aef5-m0.avro

warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-7240979918569329194-1-ec4cbc6d-78e5-47ca-b0cd-480abd39aef5.avro



Showing Metadata Files (JSON)

In [25]:
import pandas as pd

print("Showing " + metadata_file_list[0])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[0]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-bebe7ef5-5289-4cad-981d-6bb461634c57.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693794404641,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, copy-on-write, copy-on-write, co...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],63bffafe-89b6-4dc7-91fa-e0698c4347eb


In [26]:
print("Showing " + metadata_file_list[1])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[1]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-1ba964c5-4142-4d48-a738-fb31d408c861.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,refs,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,7240979918569329194,0,0,2,3,1000,1,1693794413905,s3a://go01-demo/warehouse/tablespace/external/...,[(s3a://go01-demo/warehouse/tablespace/externa...,"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, copy-on-write, copy-on-write, co...","((7240979918569329194, branch),)","[([Row(id=1, name='coffee_id', required=False,...","[(7240979918569329194, 1693794413905)]",[(s3a://go01-demo/warehouse/tablespace/externa...,"[([], 0)]",[],63bffafe-89b6-4dc7-91fa-e0698c4347eb


Showing Manifest List (AVRO - prefixed by "SNAP")

In [27]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-7240979918569329194-1-ec4cbc6d-78e5-47ca-b0cd-480abd39aef5.avro


,manifest_path,manifest_length,partition_spec_id,content,sequence_number,min_sequence_number,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,existing_rows_count,deleted_rows_count,partitions
0,s3a://go01-demo/warehouse/tablespace/external/...,7071,0,0,1,1,7240979918569329194,1,0,0,1,0,0,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"


Showing Manifest Files (Avro) i.e. which shows data file locations according to each snapshot_id

In [28]:
print("Showing " + metadata_file_list[2])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/ec4cbc6d-78e5-47ca-b0cd-480abd39aef5-m0.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,7240979918569329194,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


Show all data files post insert:

In [29]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()

,content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id
0,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,976,"{1: 33, 2: 34, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0


In [30]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()['file_path'][0]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-32d28f6b-258a-4faf-b1ab-2a74c506d427-00001.parquet'

### Table Merge Into

Create a staging table

In [31]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffee_staging PURGE")

DataFrame[]

In [32]:
spark.sql("CREATE TABLE IF NOT EXISTS lakehouse.coffee_staging\
            (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
            USING iceberg\
            PARTITIONED BY (months(coffee_sale_ts))")

DataFrame[]

In [33]:
spark.sql("INSERT INTO lakehouse.coffee_staging\
            VALUES (1, 'venti', cast(date_format('2023-07-01 11:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (2, 'grande', cast(date_format('2023-07-01 11:10:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (3, 'tall', cast(date_format('2023-04-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

#Row: Coffee_id = 1, coffee_size = venti, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 2, coffee_size = grande, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 3, coffee_size = tall, coffee_sale_ts = 2023-04-01

DataFrame[]

Merge Into Customers Table

In [34]:
spark.sql("MERGE INTO lakehouse.coffees_table c\
            USING (SELECT * FROM lakehouse.coffee_staging) s\
            ON c.coffee_id = s.coffee_id \
            WHEN MATCHED THEN UPDATE SET c.coffee_size = s.coffee_size\
            WHEN NOT MATCHED THEN INSERT *")

23/09/04 02:27:18 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


DataFrame[]

In [35]:
spark.sql("SELECT * FROM lakehouse.coffees_table.snapshots;").toPandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2023-09-04 02:26:53.905,7240979918569329194,NaN,append,s3a://go01-demo/warehouse/tablespace/external/...,{'spark.app.id': 'spark-application-1693794355...
1,2023-09-04 02:27:21.166,4555349104250238472,7.240980e+18,overwrite,s3a://go01-demo/warehouse/tablespace/external/...,"{'added-data-files': '3', 'total-equality-dele..."


In [36]:
spark.sql("SELECT * FROM lakehouse.coffees_table.manifests;").toPandas()

,content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,0,s3a://go01-demo/warehouse/tablespace/external/...,7212,0,4555349104250238472,3,0,0,0,0,0,"[(False, False, 2023-04, 2023-09)]"
1,0,s3a://go01-demo/warehouse/tablespace/external/...,7072,0,4555349104250238472,0,0,1,0,0,0,"[(False, False, 2023-09, 2023-09)]"


In [37]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()

,content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id
0,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,976,"{1: 33, 2: 34, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0
1,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(642,)",1,999,"{1: 39, 2: 41, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...","{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...",None,[4],None,0
2,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,992,"{1: 39, 2: 40, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0
3,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(639,)",1,985,"{1: 39, 2: 39, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [116, 97, 108...","{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [116, 97, 108...",None,[4],None,0


In [38]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()['file_path'][0]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-32d28f6b-258a-4faf-b1ab-2a74c506d427-00001.parquet'

In [39]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()['file_path'][1]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-07/00011-835-a9016d9a-aaae-4437-ad99-1eae8c1a81af-00001.parquet'

In [40]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()['file_path'][2]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-865-12fb383d-d126-49dc-a861-4a3750846089-00001.parquet'

In [41]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()['file_path'][3]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-04/00156-921-bd2fb5db-41f4-41b7-b873-208904c16e98-00001.parquet'

RECAP: The target table coffee was empty. We ran an insert and that created one new metadata file, one new manifest file, and one new data file.
Then we ran a Merge Into from the staging table. Of the three rows that we compared against, one was a match and two were not.
The one that was a match was rewritten to a brand new data file in the same partition (coffee_sale_ts_month=2023-09) which now has two data files.
The remaining two rows that were not a match were written to two new data files in the two respective partitions. If they had fallen under the same partition they may have been written into the same data file (next example).

#### There is a new metadata file (json) prefixed by 0002.

#### There is a new manifest list file (avro) prefixed by "snap"

#### There is a new manifest file (avro)

In [42]:
import boto3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    #print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)
    
print(*metadata_file_list, sep = "\n")

print("There is a total of " + str(len(metadata_file_list)) + " files in the Metadata layer")

Current Metadata Files: 

warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-bebe7ef5-5289-4cad-981d-6bb461634c57.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-1ba964c5-4142-4d48-a738-fb31d408c861.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-5929529c-813d-45fe-a795-44b7b5a1133c.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m0.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/ec4cbc6d-78e5-47ca-b0cd-480abd39aef5-m0.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-4555349104250238472-1-8d1438d9-9583-4820-9c1b-8faad3767db1.avro
warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-7240979918569329194-1-ec4c

Showing Latest (Current) Metadata File (JSON)

In [43]:
print("Showing " + metadata_file_list[2])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[0]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-5929529c-813d-45fe-a795-44b7b5a1133c.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-sequence-number,last-updated-ms,location,metadata-log,partition-specs,properties,schemas,snapshot-log,snapshots,sort-orders,statistics,table-uuid
0,0,-1,0,0,2,3,1000,0,1693794404641,s3a://go01-demo/warehouse/tablespace/external/...,[],"[([Row(field-id=1000, name='coffee_sale_ts_mon...","(pauldefusco, copy-on-write, copy-on-write, co...","[([Row(id=1, name='coffee_id', required=False,...",[],[],"[([], 0)]",[],63bffafe-89b6-4dc7-91fa-e0698c4347eb


Showing Latest Manifest List (AVRO - prefixed by "SNAP")

In [44]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-4555349104250238472-1-8d1438d9-9583-4820-9c1b-8faad3767db1.avro


,manifest_path,manifest_length,partition_spec_id,content,sequence_number,min_sequence_number,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,existing_rows_count,deleted_rows_count,partitions
0,s3a://go01-demo/warehouse/tablespace/external/...,7212,0,0,2,2,4555349104250238472,3,0,0,3,0,0,"[(False, False, [127, 2, 0, 0], [132, 2, 0, 0])]"
1,s3a://go01-demo/warehouse/tablespace/external/...,7072,0,0,2,2,4555349104250238472,0,0,1,0,0,1,"[(False, False, [132, 2, 0, 0], [132, 2, 0, 0])]"


In [47]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()['manifest_path'][0]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-4555349104250238472-1-8d1438d9-9583-4820-9c1b-8faad3767db1.avro


's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro'

In [48]:
print("Showing " + metadata_file_list[6])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()['manifest_path'][1]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-4555349104250238472-1-8d1438d9-9583-4820-9c1b-8faad3767db1.avro


's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m0.avro'

Showing Manifest Files (Avro) i.e. list of table partitions mapped to snapshot ID

In [49]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m0.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,2,4555349104250238472,1,1,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


In [50]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."
1,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."
2,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


In [51]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro


,status,snapshot_id,sequence_number,file_sequence_number,data_file
0,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."
1,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."
2,1,4555349104250238472,NaN,NaN,"(0, s3a://go01-demo/warehouse/tablespace/exter..."


In [52]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][0]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro


Row(content=0, file_path='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-07/00011-835-a9016d9a-aaae-4437-ad99-1eae8c1a81af-00001.parquet', file_format='PARQUET', partition=Row(coffee_sale_ts_month=642), record_count=1, file_size_in_bytes=999, column_sizes=[Row(key=1, value=39), Row(key=2, value=41), Row(key=3, value=39)], value_counts=[Row(key=1, value=1), Row(key=2, value=1), Row(key=3, value=1)], null_value_counts=[Row(key=1, value=0), Row(key=2, value=0), Row(key=3, value=0)], nan_value_counts=[], lower_bounds=[Row(key=1, value=bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'grande')), Row(key=3, value=bytearray(b'\x00ro\xf9j\xff\x05\x00'))], upper_bounds=[Row(key=1, value=bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'grande')), Row(key=3, value=bytearray(b'\x00ro\xf9j\xff\x05\x00'))], key_metadata=None, split_offsets=[4], equality_ids=None, sort_order_id=0)

In [57]:
data_file_path = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][0][1]
spark.read.parquet(data_file_path).show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        2|     grande|2023-07-01 11:10:00|
+---------+-----------+-------------------+



In [58]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][1]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro


Row(content=0, file_path='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-865-12fb383d-d126-49dc-a861-4a3750846089-00001.parquet', file_format='PARQUET', partition=Row(coffee_sale_ts_month=644), record_count=1, file_size_in_bytes=992, column_sizes=[Row(key=1, value=39), Row(key=2, value=40), Row(key=3, value=39)], value_counts=[Row(key=1, value=1), Row(key=2, value=1), Row(key=3, value=1)], null_value_counts=[Row(key=1, value=0), Row(key=2, value=0), Row(key=3, value=0)], nan_value_counts=[], lower_bounds=[Row(key=1, value=bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'venti')), Row(key=3, value=bytearray(b'\x00\xc8A9I\x04\x06\x00'))], upper_bounds=[Row(key=1, value=bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'venti')), Row(key=3, value=bytearray(b'\x00\xc8A9I\x04\x06\x00'))], key_metadata=None, split_offsets=[4], equality_ids=None, sort_order_id=0)

In [59]:
data_file_path = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][1][1]
spark.read.parquet(data_file_path).show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|      venti|2023-09-01 10:00:00|
+---------+-----------+-------------------+



In [60]:
print("Showing " + metadata_file_list[4])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][2]

Showing warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/8d1438d9-9583-4820-9c1b-8faad3767db1-m1.avro


Row(content=0, file_path='s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-04/00156-921-bd2fb5db-41f4-41b7-b873-208904c16e98-00001.parquet', file_format='PARQUET', partition=Row(coffee_sale_ts_month=639), record_count=1, file_size_in_bytes=985, column_sizes=[Row(key=1, value=39), Row(key=2, value=39), Row(key=3, value=39)], value_counts=[Row(key=1, value=1), Row(key=2, value=1), Row(key=3, value=1)], null_value_counts=[Row(key=1, value=0), Row(key=2, value=0), Row(key=3, value=0)], nan_value_counts=[], lower_bounds=[Row(key=1, value=bytearray(b'\x03\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'tall')), Row(key=3, value=bytearray(b'\x007D\x14E\xf8\x05\x00'))], upper_bounds=[Row(key=1, value=bytearray(b'\x03\x00\x00\x00\x00\x00\x00\x00')), Row(key=2, value=bytearray(b'tall')), Row(key=3, value=bytearray(b'\x007D\x14E\xf8\x05\x00'))], key_metadata=None, split_offsets=[4], equality_ids=None, sort_order_id=0)

In [61]:
data_file_path = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[4]).toPandas()['data_file'][2][1]
spark.read.parquet(data_file_path).show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        3|       tall|2023-04-01 12:01:00|
+---------+-----------+-------------------+



### Time Travel 

In [ ]:
snapshots_df = spark.sql("SELECT * FROM lakehouse.customer_table.snapshots;")

In [ ]:
first_snapshot = snapshots_df.select("snapshot_id").head(1)[0][0]

#### Validate that the output dataframe only includes one row per the original insert

In [ ]:
spark.read\
    .option("snapshot-id", first_snapshot)\
    .format("iceberg")\
    .load("lakehouse.customer_table").toPandas()

In [ ]:
avro_tempdf = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[6]).toPandas()

In [ ]:
avro_tempdf.columns

In [ ]:
avro_tempdf['partitions']

In [ ]:
avro_tempdf['added_rows_count']

In [ ]:
avro_tempdf['existing_rows_count']

In [ ]:
avro_tempdf['added_data_files_count']

In [ ]:
print("Showing " + metadata_file_list[2])
json_tempdf = spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

In [ ]:
json_tempdf.columns

In [ ]:
json_tempdf['current-schema-id']

In [ ]:
list(json_tempdf['snapshots'])

In [ ]:
json_tempdf['partition-spec']

In [ ]:
list(json_tempdf['partition-specs'])

In [ ]:
spark.sql("SELECT * FROM lakehouse.coffees_table_2.all_data_files;").show()

### Partition Evolution

Spark partitioning is a way to split the data into multiple partitions so that you can execute transformations on multiple partitions in parallel which allows completing the job faster. You can also write partitioned data into a file system (multiple sub-directories) for faster reads by downstream systems.

Spark has several partitioning methods to achieve parallelism, based on your need, you should choose which one to use.

Creating New Data to Test Partition Evolution

In [ ]:
from pyspark.sql.types import LongType, IntegerType, StringType

import dbldatagen as dg

shuffle_partitions_requested = 20
device_population = 100000
data_rows = 20 * 1000000
#partitions_requested = 20

spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

country_codes = [
    "CN", "US", "FR", "CA", "IN", "JM", "IE", "PK", "GB", "IL", "AU", 
    "SG", "ES", "GE", "MX", "ET", "SA", "LB", "NL", "IT"
]
#country_weights = [
#    1300, 365, 67, 38, 1300, 3, 7, 212, 67, 9, 25, 6, 47, 83, 
#    126, 109, 58, 8, 17,
#]

manufacturers = [
    "Delta corp", "Xyzzy Inc.", "Lakehouse Ltd", "Acme Corp", "Embanks Devices",
]

lines = ["delta", "xyzzy", "lakehouse", "gadget", "droid"]

testDataSpec = (
    dg.DataGenerator(spark, name="device_data_set", rows=data_rows) 
                     #,partitions=partitions_requested)
    .withIdOutput()
    # we'll use hash of the base field to generate the ids to
    # avoid a simple incrementing sequence
    .withColumn("internal_device_id", "long", minValue=0x1000000000000, 
                uniqueValues=device_population, omit=True, baseColumnType="hash",
    )
    # note for format strings, we must use "%lx" not "%x" as the
    # underlying value is a long
    .withColumn(
        "device_id", "string", format="0x%013x", baseColumn="internal_device_id"
    )
    # the device / user attributes will be the same for the same device id
    # so lets use the internal device id as the base column for these attribute
    .withColumn("country", "string", values=country_codes, #weights=country_weights, 
                baseColumn="internal_device_id")
    .withColumn("manufacturer", "string", values=manufacturers, 
                baseColumn="internal_device_id", )
    # use omit = True if you don't want a column to appear in the final output
    # but just want to use it as part of generation of another column
    .withColumn("line", "string", values=lines, baseColumn="manufacturer", 
                baseColumnType="hash", omit=True )
    .withColumn("model_ser", "integer", minValue=1, maxValue=11, baseColumn="device_id", 
                baseColumnType="hash", omit=True, )
    .withColumn("model_line", "string", expr="concat(line, '#', model_ser)", 
                baseColumn=["line", "model_ser"] )
    .withColumn("event_type", "string", 
                values=["activation", "deactivation", "plan change", "telecoms activity", 
                        "internet activity", "device error", ],
                random=True)
    .withColumn("event_ts", "timestamp", begin="2020-01-01 01:00:00", 
                end="2020-12-31 23:59:00", 
                interval="1 minute", random=True )
)

dfTestData = testDataSpec.build()

display(dfTestData)

In [ ]:
dfTestData.head()

In [ ]:
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

In [ ]:
spark.sql("DROP TABLE IF EXISTS spark_catalog.lakehouse.partition_evol_tbl PURGE")

In [ ]:
#dfTestData.groupBy("country").count().show()

In [ ]:
#dfTestData.rdd.getNumPartitions()

Iceberg requires the data to be sorted according to the partition spec per task (Spark partition) in prior to write against partitioned table. This applies both Writing with SQL and Writing with DataFrames.

In [ ]:
dfTestData.sortWithinPartitions("country").writeTo("spark_catalog.lakehouse.p_evol_tbl").partitionedBy("country").using("iceberg").create()#.append()#replace()#overwritePartitions()#create()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.PARTITIONS").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.files").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.manifests").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.all_manifests").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.all_data_files").show()

In [ ]:
#spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.snapshots").show()

Adding a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout. Old data files will have null values for the new partition fields in metadata tables.

In [ ]:
print("TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

In [ ]:
print("ADD PARTITION BY EVENT TIMESTAMP MONTHS: ")
print("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
spark.sql("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
#spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl REPLACE PARTITION FIELD hours(dob) WITH state")
#spark.sql("ALTER TABLE prod.db.sample ADD PARTITION FIELD month")

#ALTER TABLE spark_catalog.lakehouse.part_evol_tbl ADD PARTITION FIELD days(event_ts)

In [ ]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

In [ ]:
appendDf = dfTestData.sample(fraction=0.3, seed=3)

In [ ]:
appendDf.dtypes

In [ ]:
appendDf.rdd.getNumPartitions()

In [ ]:
appendDf.show()

In [ ]:
appendDf.sortWithinPartitions("country").show()

In [ ]:
#appendDf.sortWithinPartitions("country", "month(event_ts)").show()

In [ ]:
appendDf.sortWithinPartitions("country").writeTo("spark_catalog.lakehouse.p_evol_tbl").using("iceberg").append() #.append()#replace()#overwritePartitions()#create()

In [ ]:
print("TABLE PARTITIONS AFTER APPEND: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show(100)

Dropping a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout.

In [ ]:
spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl DROP PARTITION FIELD bucket(16, device_id)")

In [ ]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.PARTITIONS").show()

##### Only json files have been added (one per each time you repartitioned) but Avro files have stayed the same

In [ ]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    #print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)
    
metadata_file_list

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS customer_table (id BIGINT, state STRING, country STRING, dob TIMESTAMP) USING iceberg PARTITIONED BY ( hours(dob))")

In [ ]:
spark.sql("SELECT HOUR(dob) FROM spark_catalog.lakehouse.customer_table").show()

In [ ]:
spark.sql("SELECT DAY(dob) FROM spark_catalog.lakehouse.customer_table").show()

### Dropping Tables

In [ ]:
spark.sql("DROP TABLE IF EXISTS lakehouse.staging")

Validate that the metadata folder is now empty but the data folder still retains parquet files.

![alt text](../img/s3_droptable_1.png)

![alt text](../img/s3_droptable_2.png)

![alt text](../img/s3_droptable_3.png)

In [ ]:
spark.sql("ALTER TABLE lakehouse.customers_table\
            SET TBLPROPERTIES ('format-version' = '2')")

In [ ]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

for object_summary in my_bucket.objects.filter(Prefix=metadata_path):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

In [ ]:
print("Showing " + metadata_file_list[3])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[3]).toPandas()